Principalement basé sur l’enquête les origines du populisme : enquête sur un schisme politique et social de Daniel Cohen, Martial Foucault et Yann Algan, notre projet tente d’expliquer causalement le populisme de droite, à la fois de manière objective et subjective. Le livre référant est une vaste enquête électorale effectuée à l’échelle européenne essayant de dépasser les anciennes explications causales du vote populiste, notamment en utilisant des variables subjectives. Prenant comme base de données celle utilisée par les chercheurs, c’est à dire « l’european social survey -qui est une enquête faite à l’échelle européenne faite tous les deux dans 35 pays différents- nous avons à notre disposition un nombre importants de variables, à la fois objectives et subjectives, pouvant être regroupées en 8 catégories différentes : confiance sociale, politique, bien être subjectif, changement climatique, bien être objectif, caractéristiques sociodémographiques, caractéristiques du répondant et valeurs de l’individu.  

Dans notre projet, nous restreignons l’étude à celle du vote populiste de droite en France en 2016. Nous caractérisons le populisme comme un mouvement de rejet des élites traditionnelles et d’un ordre établi, c’est-à-dire des forces antisystèmes ayant comme dénominateur commun l’insécurité économiques notamment. Cependant, l’objet de notre étude se restreint au populisme de droite. Le populisme de gauche pouvant être caractérisée par une radicalisation de la pensée de la gauche dite classique (ayant grossièrement pour projet principalement la réduction des inégalités).  Suite à la dissolution de la société industrielle dite classistes -pouvant être représentée schématiquement par une opposition binaire de deux classes sociales qui se font face- la société post-industrielle émerge et tend à dissoudre l’individu dans une société où il peut peiner à retrouver institutions socialisantes qui ont forgé son identité. Notre projet part de ce constat, et des intuitions offertes par le livre, qui tend à expliquer le vote populiste par un rapport à l’altérité brisé, un rapport à l’autre dissolu. On va ainsi tenter d’expliquer le vote populiste de droit notamment à l’aune de cette rupture  anthropologique pouvant être modéliser par un proxy de la confiance que l’on tentera de construire.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels as sm
import statsmodels.api as st
import statsmodels.formula.api as smf
import urllib
import bs4